In [3]:
import ast
import glob
import warnings
from collections import defaultdict
from datetime import date

import numpy as np
import pandas as pd
import wandb
import matplotlib.pyplot as plt

PROJ_PREFIX = "main_exp"
PROJ_TYPES = ["SANN", "GPSE"]
PROJ_DS = [
    "NCI1",
    "NCI109",
    "MUTAG",
    "PROTEINS",
    "ZINC",
    "IMDB-BINARY",
    "IMDB-MULTI",
]

today = date.today()
api = wandb.Api(overrides={"base_url": "https://api.wandb.ai"})

In [ ]:
# # Find all csv files in the current directory
csv_files = glob.glob("csv/*.csv")
# # Collect all the names of the csv files without the extension
csv_names = [csv_file[:-4] for csv_file in csv_files]
user = "telyatnikov_sap"

for project_dataset in PROJ_DS:
    for project_type in PROJ_TYPES:
        project_name = f"{PROJ_PREFIX}_{project_type}_{project_dataset}"
        if project_name not in csv_names:
            runs = api.runs(f"{user}/{project_name}")
            try:
                list(runs)
            except:
                continue

            summary_list, config_list, name_list = [], [], []
            for run in runs:
                # .summary contains the output keys/values for metrics like accuracy.
                #  We call ._json_dict to omit large files
                summary_list.append(run.summary._json_dict)

                # .config contains the hyperparameters.
                #  We remove special values that start with _.
                config_list.append(
                    {
                        k: v
                        for k, v in run.config.items()
                        if not k.startswith("_")
                    }
                )

                # .name is the human-readable name of the run.
                name_list.append(run.name)

            runs_df = pd.DataFrame(
                {
                    "summary": summary_list,
                    "config": config_list,
                    "name": name_list,
                }
            )

            runs_df.to_csv(f"csv/{user}_{project_name}.csv")
        else:
            runs_df = pd.read_csv(
                f"csv/{user}_{project_name}.csv", index_col=0
            )

            for row in runs_df.iloc:
                row["summary"] = ast.literal_eval(row["summary"])
                row["config"] = ast.literal_eval(row["config"])
    for row in runs_df.iloc:
        row["summary"].update(row["config"])

    lst = [i["summary"] for i in runs_df.iloc]
    df = pd.DataFrame.from_dict(lst)

    df_init = df.copy()

In [4]:
csv_files = glob.glob("csv/*.csv")
df_list = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file, index_col=0)
    for row in df.iloc:
        row["summary"] = ast.literal_eval(row["summary"])
        row["config"] = ast.literal_eval(row["config"])
        row["summary"].update(row["config"])

    lst = [i["summary"] for i in df.iloc]
    df = pd.DataFrame.from_dict(lst)

    df_init = df.copy()

    df_list.append(df_init)

df = pd.concat(df_list, ignore_index=True)

In [5]:
def normalize_column(df, column_to_normalize):
    # Use json_normalize to flatten the nested dictionaries into separate columns
    flattened_df = pd.json_normalize(df[column_to_normalize])
    # Rename columns to include 'nested_column' prefix
    flattened_df.columns = [
        f"{column_to_normalize}.{col}" for col in flattened_df.columns
    ]
    # Concatenate the flattened DataFrame with the original DataFrame
    result_df = pd.concat([df, flattened_df], axis=1)
    # Get new columns names
    new_columns = flattened_df.columns
    # Drop the original nested column if needed
    result_df.drop(column_to_normalize, axis=1, inplace=True)
    return result_df, new_columns


# Config columns to normalize
columns_to_normalize = [
    "model",
    "dataset",
    "callbacks",
    "paths",
    "transforms",
    "optimizer",
]

# Keep track of config columns added
config_columns = []
for column in columns_to_normalize:
    df, columns = normalize_column(df, column)
    config_columns.extend(columns)

In [93]:
df = pd.read_csv("merged_csv/merged_normalized.csv")
columns_to_eval = ["transforms.sann_encoding.pe_types"]
for col in columns_to_eval:
    df[col] = df[col].apply(lambda x: str(x).replace("nan", "None"))
    df[col] = df[col].apply(ast.literal_eval)

/var/folders/hp/0rz4lf5s51bfpr4hwz5hx0kh0000gn/T/ipykernel_5972/1307283954.py:1: DtypeWarning: Columns (35,37,92,113,123,132,134,135,136,148,157,159,160,162,163,164,166,168,169,170,173,174,175,181,182,183,188,190,191,192,193,195,196,197,199,200,201,203,204,206,207,208,209,210,212,213,214,216,217,218,219,221,222,223,224,225,226,227,228,230,231,232,233,234,235,236,248,250,254,261,264,267,270) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_csv/merged_normalized.csv')


# Correct model name based on each pre-trained model

In [94]:
def map_name(row):
    if isinstance(row["transforms.sann_encoding.pe_types"], list):
        return "HOPSE_MANUAL_PE"
    elif row["model.model_name"] == "sann":
        if type(
            row["transforms.sann_encoding.pretrain_model"]
        ) == float and pd.isna(row["transforms.sann_encoding.pretrain_model"]):
            return "SANN"
        else:
            return "HOPSE_GPSE"
    else:
        return row["model.model_name"]

In [95]:
df["model.model_name"] = df.apply(map_name, axis=1)
df["transforms.sann_encoding.neighborhoods"] = df[
    "transforms.sann_encoding.neighborhoods"
].astype(str)

In [82]:
df["model.model_name"].unique()

array(['HOPSE_GPSE', 'scn', 'sccnn', nan, 'SANN', 'HOPSE_MANUAL_PE',
       'gin', 'gcn'], dtype=object)

In [96]:
df["dataset.loader.parameters.data_name"].unique()

array(['NCI109', 'MANTRA_betti_numbers', 'NCI1', 'MUTAG', 'MANTRA_name',
       'MANTRA_orientation', nan, 'PROTEINS', 'IMDB-BINARY', 'ZINC',
       'IMDB-MULTI'], dtype=object)

In [229]:
df[df["dataset.loader.parameters.data_name"] == "MANTRA_betti_numbers"]

,Unnamed: 0.1,Unnamed: 0,AvgTime/train_batch_mean,AvgTime/train_batch_std,AvgTime/train_epoch_mean,AvgTime/train_epoch_std,AvgTime/val_batch_mean,AvgTime/val_batch_std,AvgTime/val_epoch_mean,AvgTime/val_epoch_std,...,callbacks.model_checkpoint.save_last,callbacks.model_checkpoint.save_top_k,callbacks.model_checkpoint.every_n_epochs,callbacks.model_checkpoint.save_weights_only,callbacks.model_checkpoint.every_n_train_steps,callbacks.model_checkpoint.train_time_interval,callbacks.model_checkpoint.auto_insert_metric_name,callbacks.model_checkpoint.save_on_train_epoch_end,callbacks.learning_rate_monitor._target_,callbacks.learning_rate_monitor.logging_interval
15348,15348,15348,0.028011,0.001779,18.782126,3.057649,0.013052,0.000510,7.468087,0.636926,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch
15349,15349,15349,0.028702,0.001932,19.219027,3.050732,0.013355,0.000435,7.539920,0.113171,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch
15350,15350,15350,0.027799,0.001745,18.830744,3.077185,0.013167,0.000516,7.427293,0.152190,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch
15351,15351,15351,0.028366,0.001876,19.200341,3.248864,0.013314,0.000443,7.667151,0.691265,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch
15352,15352,15352,0.045146,0.016090,18.786867,3.569898,0.024431,0.011655,7.799555,1.317402,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194266,194266,194266,0.016987,0.001852,10.728102,2.240022,0.009448,0.001915,5.672234,1.296537,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch
194267,194267,194267,0.016992,0.001949,10.429274,2.151637,0.009495,0.002006,4.808242,1.162993,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch
194268,194268,194268,0.017494,0.002339,11.165850,2.520713,0.009686,0.002220,5.874798,1.805193,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch
194269,194269,194269,0.016411,0.001949,9.412992,1.910430,0.009065,0.001948,4.128219,0.884713,...,False,1.0,NaN,False,NaN,NaN,False,NaN,lightning.pytorch.callbacks.LearningRateMonitor,epoch


In [51]:
[i for i in df.columns]

['Unnamed: 0.1',
 'Unnamed: 0',
 'AvgTime/train_batch_mean',
 'AvgTime/train_batch_std',
 'AvgTime/train_epoch_mean',
 'AvgTime/train_epoch_std',
 'AvgTime/val_batch_mean',
 'AvgTime/val_batch_std',
 'AvgTime/val_epoch_mean',
 'AvgTime/val_epoch_std',
 '_runtime',
 '_step',
 '_timestamp',
 '_wandb',
 'epoch',
 'lr-Adam',
 'test/accuracy',
 'test/auroc',
 'test/loss',
 'test/precision',
 'test/recall',
 'train/accuracy',
 'train/auroc',
 'train/loss',
 'train/precision',
 'train/recall',
 'trainer/global_step',
 'val/accuracy',
 'val/auroc',
 'val/loss',
 'val/precision',
 'val/recall',
 'loss',
 'seed',
 'tags',
 'test',
 'paths',
 'train',
 'extras',
 'logger',
 'trainer',
 'ckpt_path',
 'evaluator',
 'task_name',
 'model/params/total',
 'model/params/trainable',
 'model/params/non_trainable',
 'test/accuracy_0',
 'test/accuracy_1',
 'test/accuracy_2',
 'test/f1_0',
 'test/f1_1',
 'test/f1_2',
 'test/precision_0',
 'test/precision_1',
 'test/precision_2',
 'test/recall_0',
 'test/reca

### Get grouped df

In [97]:
df = df[~(df["dataset.split_params.data_seed"].isna())]

30996    0.829787
30997    0.914894
30998    0.851064
30999    0.872340
31000    0.893617
           ...   
46351    0.851064
46352    0.957447
46353    0.638298
46354    0.638298
46355    0.744681
Name: test/accuracy, Length: 15360, dtype: float64

In [9]:
df["model.model_domain"].unique()

array(['cell', 'simplicial', 'graph'], dtype=object)

In [54]:
list(df.columns)

['Unnamed: 0.1',
 'Unnamed: 0',
 'AvgTime/train_batch_mean',
 'AvgTime/train_batch_std',
 'AvgTime/train_epoch_mean',
 'AvgTime/train_epoch_std',
 'AvgTime/val_batch_mean',
 'AvgTime/val_batch_std',
 'AvgTime/val_epoch_mean',
 'AvgTime/val_epoch_std',
 '_runtime',
 '_step',
 '_timestamp',
 '_wandb',
 'epoch',
 'lr-Adam',
 'test/accuracy',
 'test/auroc',
 'test/loss',
 'test/precision',
 'test/recall',
 'train/accuracy',
 'train/auroc',
 'train/loss',
 'train/precision',
 'train/recall',
 'trainer/global_step',
 'val/accuracy',
 'val/auroc',
 'val/loss',
 'val/precision',
 'val/recall',
 'loss',
 'seed',
 'tags',
 'test',
 'paths',
 'train',
 'extras',
 'logger',
 'trainer',
 'ckpt_path',
 'evaluator',
 'task_name',
 'model/params/total',
 'model/params/trainable',
 'model/params/non_trainable',
 'test/accuracy_0',
 'test/accuracy_1',
 'test/accuracy_2',
 'test/f1_0',
 'test/f1_1',
 'test/f1_2',
 'test/precision_0',
 'test/precision_1',
 'test/precision_2',
 'test/recall_0',
 'test/reca

In [219]:
changed_params = []
for param in list(df.columns):
    if df[param].dtype == pd.CategoricalDtype:
        continue
    if len(df[param].unique()) > 1:
        changed_params.append(param)

In [220]:
changed_params

['Unnamed: 0.1',
 'Unnamed: 0',
 'AvgTime/train_batch_mean',
 'AvgTime/train_batch_std',
 'AvgTime/train_epoch_mean',
 'AvgTime/train_epoch_std',
 'AvgTime/val_batch_mean',
 'AvgTime/val_batch_std',
 'AvgTime/val_epoch_mean',
 'AvgTime/val_epoch_std',
 '_runtime',
 '_step',
 '_timestamp',
 'epoch',
 'lr-Adam',
 'test/accuracy',
 'test/auroc',
 'test/loss',
 'test/precision',
 'test/recall',
 'train/accuracy',
 'train/auroc',
 'train/loss',
 'train/precision',
 'train/recall',
 'trainer/global_step',
 'val/accuracy',
 'val/auroc',
 'val/loss',
 'val/precision',
 'val/recall',
 'seed',
 'model/params/total',
 'model/params/trainable',
 'model/params/non_trainable',
 'test/accuracy_0',
 'test/accuracy_1',
 'test/accuracy_2',
 'test/f1_0',
 'test/f1_1',
 'test/f1_2',
 'test/precision_0',
 'test/precision_1',
 'test/precision_2',
 'test/recall_0',
 'test/recall_1',
 'test/recall_2',
 'train/accuracy_0',
 'train/accuracy_1',
 'train/accuracy_2',
 'train/f1_0',
 'train/f1_1',
 'train/f1_2',
 

In [37]:
df["model.model_name"].unique()

array(['HOPSE_GPSE', 'scn', 'sccnn', 'SANN', 'gin', 'gcn'], dtype=object)

In [98]:
# Extract best results for each model and dataset
# 1. Keep the columns that are necessary for the comparison

sweeped_columns = [
    "transforms.sann_encoding.max_hop",
    "transforms.sann_encoding.max_rank",
    "transforms.sann_encoding.neighborhoods",
    "transforms.sann_encoding.pretrain_model",
    "transforms.sann_encoding.complex_dim",
    "model.feature_encoder.proj_dropout",
    "model.backbone.n_layers",
    "model.backbone.hidden_channels",
    "model.readout.hidden_dim",
    "model.feature_encoder.out_channels",
    # SCCNN
    "model.backbone.sc_order",
    "model.backbone.conv_order",
    "model.readout.readout_name",
    # Others
    "optimizer.parameters.weight_decay",
    "optimizer.parameters.lr",
    "dataset.dataloader_params.batch_size",
    "dataset.loader.parameters.manifold_dim",
    # Additional
    "transforms.sann_encoding.copy_initial",
    "transforms.graph2cell_lifting.max_cell_length",
    "transforms.sann_encoding.use_initial_features",
    "transforms.sann_encoding.is_undirected",
    "transforms.sann_encoding.target_pe_dim",
    "transforms.sann_encoding.laplacian_norm_type",
    "transforms.sann_encoding.posenc_LapPE_eigen_max_freqs",
    "transforms.sann_encoding.posenc_LapPE_eigen_eigvec_abs",
    "transforms.sann_encoding.posenc_LapPE_eigen_eigvec_norm",
    "transforms.sann_encoding.posenc_LapPE_eigen_skip_zero_freq",
    "transforms.redefine_simplicial_neighborhoods.signed",
    "transforms.redefine_simplicial_neighborhoods.complex_dim",
]
run_columns = [
    "dataset.split_params.data_seed",
    "seed",
]

# Dataset and model columns
dataset_model_columns = [
    "model.model_name",
    "model.model_domain",
    "dataset.loader.parameters.data_name",
]

# Performance columns
performance_columns = [
    "val/loss",
    "test/loss",
    "val/mae",
    "test/mae",
    "val/mse",
    "test/mse",
    "val/accuracy",
    "test/accuracy",
    "val/auroc",
    "test/auroc",
    "val/recall",
    "test/recall",
    "val/precision",
    "test/precision",
    "val/f1",
    "test/f1",
]
keep_columns = (
    dataset_model_columns + sweeped_columns + performance_columns + run_columns
)
df_keep = df[keep_columns]

In [103]:
df_keep[
    (df_keep["model.model_name"] == "HOPSE_GPSE")
    & (df["dataset.loader.parameters.data_name"] == "MUTAG")
    & (df["model.model_domain"] == "simplicial")
]["test/accuracy"]

Series([], Name: test/accuracy, dtype: float64)

In [91]:
def map_correct_dataname(row):
    if row["dataset.loader.parameters.data_name"] == "MANTRA_betti_numbers":
        rows = []
        for i in range(3):
            row_dict = row.to_dict()
            row_dict["dataset.loader.parameters.data_name"] = (
                row_dict["dataset.loader.parameters.data_name"] + f"_{i}"
            )
            row_dict["val/f1"] = row_dict[f"val/f1_{i}"]
            row_dict["test/f1"] = row_dict[f"test/f1_{i}"]
            rows.append(pd.DataFrame.from_records([row_dict]))
        return pd.concat(rows)

In [177]:
scores_dfs = []
for i, row in df.iterrows():
    if row["dataset.loader.parameters.data_name"] == "MANTRA_betti_numbers":
        rows = []
        for i in range(3):
            row_dict = row.to_dict()
            row_dict["dataset.loader.parameters.data_name"] = (
                row_dict["dataset.loader.parameters.data_name"] + f"_{i}"
            )
            row_dict["val/f1"] = row_dict[f"val/f1_{i}"]
            row_dict["test/f1"] = row_dict[f"test/f1_{i}"]
            rows.append(pd.DataFrame.from_records([row_dict]))
        scores_dfs.append(pd.concat(rows))
scores_df = pd.concat(scores_dfs)

In [109]:
explode_df = df.explode("val/f1")
explode_df[~df["val/f1_0"].isna()][["val/f1", "val/f1_0"]]

/var/folders/hp/0rz4lf5s51bfpr4hwz5hx0kh0000gn/T/ipykernel_20668/2233766090.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  explode_df[~df['val/f1_0'].isna()][['val/f1', 'val/f1_0']]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [178]:
scores_df[
    ["model.model_name", "dataset.loader.parameters.data_name", "val/f1"]
]
df = df[df["dataset.loader.parameters.data_name"] != "MANTRA_betti_numbers"]
df = pd.concat([df, scores_df])

In [99]:
performance_classification = [
    "val/accuracy",
    "test/accuracy",
    "val/auroc",
    "test/auroc",
    "val/loss",
    "val/recall",
    "test/recall",
    "val/precision",
    "test/precision",
    "val/f1",
    "test/f1",
]
performance_split = [
    "val/loss",
    "test/loss",
]
performance_regression = [
    "val/mae",
    "test/mae",
    "val/mse",
    "test/mse",
]
# Define a dict of dicts for each dataset the corresponding optimization metrics
optimization_metrics = {
    "NCI109": {
        "optim_metric": "val/accuracy",
        "eval_metric": "test/accuracy",
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "NCI1": {
        "optim_metric": "val/accuracy",
        "eval_metric": "test/accuracy",
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "PROTEINS": {
        "optim_metric": "val/accuracy",
        "eval_metric": "test/accuracy",
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "MUTAG": {
        "optim_metric": "val/accuracy",
        "eval_metric": "test/accuracy",
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "ZINC": {
        "optim_metric": "val/mae",
        "eval_metric": "test/mae",
        "direction": "min",
        "performance_columns": performance_regression,
    },
    "IMDB-BINARY": {
        "optim_metric": "val/accuracy",
        "eval_metric": "test/accuracy",
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "IMDB-MULTI": {
        "optim_metric": "val/accuracy",
        "eval_metric": "test/accuracy",
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "Cora": {
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "Citeseer": {
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "PubMed": {
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "MANTRA_name": {
        "optim_metric": "val/f1",
        "eval_metric": "test/f1",
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "MANTRA_orientation": {
        "optim_metric": "val/f1",
        "eval_metric": "test/f1",
        "direction": "max",
        "performance_columns": performance_classification,
    },
    "MANTRA_betti_numbers": {
        "optim_metric": "val/loss",
        "eval_metric": "val/loss",
        "direction": "min",
        "performance_columns": performance_classification,
    },
    "MANTRA_betti_numbers_0": {
        "optim_metric": "val/loss",
        "eval_metric": "test/f1",
        "direction": "min",
        "performance_columns": performance_classification,
    },
    "MANTRA_betti_numbers_1": {
        "optim_metric": "val/loss",
        "eval_metric": "test/f1",
        "direction": "min",
        "performance_columns": performance_classification,
    },
    "MANTRA_betti_numbers_2": {
        "optim_metric": "val/loss",
        "eval_metric": "test/f1",
        "direction": "min",
        "performance_columns": performance_classification,
    },
}

len(optimization_metrics)

16

In [185]:
mask = (
    df_keep["dataset.loader.parameters.data_name"] == "MANTRA_betti_numbers_0"
)
mask = (mask) & (df_keep["model.model_name"] == "HOPSE_GPSE")
df_keep[mask][["model.model_name", "val/loss", "test/f1"]]

,model.model_name,val/loss,test/f1
0,HOPSE_GPSE,0.036120,1.0
0,HOPSE_GPSE,0.036363,1.0
0,HOPSE_GPSE,0.036407,1.0
0,HOPSE_GPSE,0.036260,1.0
0,HOPSE_GPSE,0.034499,1.0
...,...,...,...
0,HOPSE_GPSE,0.039794,1.0
0,HOPSE_GPSE,0.052050,1.0
0,HOPSE_GPSE,0.044742,1.0
0,HOPSE_GPSE,0.059122,1.0


In [19]:
collect_subsets = None

In [100]:
search_metric_types = [
    "accuracy",
]  #'auroc', 'recall', 'precision']


# Get unique datasets
datasets = list(df["dataset.loader.parameters.data_name"].unique())
# Get unique models
models = list(df["model.model_name"].unique())

best_results = defaultdict(dict)
best_results_all_metrics = defaultdict(dict)
best_runs = defaultdict(dict)
collect_subsets = defaultdict(dict)
collect_bast_parameters = defaultdict(dict)
# Got over each dataset and model and find the best result
for dataset in datasets:
    for model in models:
        # Get the subset of the DataFrame for the current dataset and model
        subset = df_keep[
            (df_keep["dataset.loader.parameters.data_name"] == dataset)
        ]

        optim_metric = optimization_metrics[dataset]["optim_metric"]
        eval_metric = optimization_metrics[dataset]["eval_metric"]
        direction = optimization_metrics[dataset]["direction"]

        # Keep metrics that matters for dataset
        performance_columns = optimization_metrics[dataset][
            "performance_columns"
        ]
        subset = subset[
            dataset_model_columns
            + sweeped_columns
            + performance_columns
            + run_columns
        ]

        aggregated = subset.groupby(
            sweeped_columns + ["model.model_name", "model.model_domain"],
            dropna=False,
        ).agg(
            {col: ["mean", "std", "count"] for col in performance_columns},
        )

        # aggregated = subset.groupby(sweeped_columns, dropna=False).count()
        n_count = 5 if "MANTRA" not in dataset else 4
        # Go from MultiIndex to Index
        aggregated = aggregated.reset_index()
        print(f"Dataset: {dataset}, Model: {model}")
        print(aggregated[(eval_metric, "count")].unique())
        # print(aggregated['dataset.split_params.data_seed'].unique())
        print(
            (aggregated[(eval_metric, "count")] >= n_count).sum()
            / len(aggregated)
            * 100
        )
        aggregated = aggregated[aggregated[(eval_metric, "count")] >= n_count]
        # print(len(aggregated[aggregated['seed'] > 4]))
        # aggregated = aggregated.sort_values(
        #         by=(eval_metric, "mean"), ascending=(direction == 'min')
        #     )

        # Git percent in case of classification
        if "test/accuracy" in performance_columns:
            # Go over all the performance columns and multiply by 100
            for col in performance_columns:
                aggregated[(col, "mean")] *= 100
                aggregated[(col, "std")] *= 100

            # Round performance columns values up to 2 decimal points
            for col in performance_columns:
                aggregated[(col, "mean")] = aggregated[(col, "mean")].round(4)
                aggregated[(col, "std")] = aggregated[(col, "std")].round(4)

        else:
            # Round all values up to 4 decimal points
            # Round performance columns values up to 4 decimal points
            for col in performance_columns:
                aggregated[(col, "mean")] = aggregated[(col, "mean")].round(4)
                aggregated[(col, "std")] = aggregated[(col, "std")].round(4)

        collect_subsets[dataset] = aggregated

Dataset: NCI109, Model: HOPSE_GPSE
[5 4 3]
99.79163772746215
Dataset: NCI109, Model: scn
[5 4 3]
99.79163772746215
Dataset: NCI109, Model: sccnn
[5 4 3]
99.79163772746215
Dataset: NCI109, Model: SANN
[5 4 3]
99.79163772746215
Dataset: NCI109, Model: HOPSE_MANUAL_PE
[5 4 3]
99.79163772746215
Dataset: NCI109, Model: gin
[5 4 3]
99.79163772746215
Dataset: NCI109, Model: gcn
[5 4 3]
99.79163772746215
Dataset: MANTRA_betti_numbers, Model: HOPSE_GPSE
[ 5  4  3 12  8  7  6]
99.93055555555556
Dataset: MANTRA_betti_numbers, Model: scn
[ 5  4  3 12  8  7  6]
99.93055555555556
Dataset: MANTRA_betti_numbers, Model: sccnn
[ 5  4  3 12  8  7  6]
99.93055555555556
Dataset: MANTRA_betti_numbers, Model: SANN
[ 5  4  3 12  8  7  6]
99.93055555555556
Dataset: MANTRA_betti_numbers, Model: HOPSE_MANUAL_PE
[ 5  4  3 12  8  7  6]
99.93055555555556
Dataset: MANTRA_betti_numbers, Model: gin
[ 5  4  3 12  8  7  6]
99.93055555555556
Dataset: MANTRA_betti_numbers, Model: gcn
[ 5  4  3 12  8  7  6]
99.930555555555

In [238]:
protein_subset = collect_subsets["PROTEINS"][
    [
        "test/accuracy",
        "val/accuracy",
        "model.model_name",
        "model.model_domain",
        "transforms.sann_encoding.neighborhoods",
        "model.backbone.n_layers",
    ]
]
protein_subset.loc[:, "overfit"] = (
    protein_subset[("val/accuracy", "mean")]
    - protein_subset[("test/accuracy", "mean")]
)
protein_subset.sort_values(by="overfit", ascending=False).head(10)

TypeError: unhashable type: 'list'

In [41]:
df_mantra_name = collect_subsets["ZINC"][
    [
        "val/mse",
        "model.model_name",
        "model.model_domain",
        "transforms.sann_encoding.neighborhoods",
        "model.backbone.n_layers",
    ]
].sort_values(by=("val/mse", "mean"), ascending=True)
df_mantra_name["model.model_name"].unique()
# df_mantra_name[df_mantra_name['model.model_name']=='HOPSE_GPSE']

array(['HOPSE_MANUAL_PE', 'HOPSE_GPSE', 'SANN'], dtype=object)

In [109]:
df_test = collect_subsets["MANTRA_name"][
    collect_subsets["MANTRA_name"]["model.model_name"] == "HOPSE_MANUAL_PE"
][
    [
        "test/accuracy",
        "val/accuracy",
        "model.model_name",
        "transforms.sann_encoding.max_rank",
        "model.model_domain",
        "transforms.sann_encoding.neighborhoods",
        "model.backbone.n_layers",
        "optimizer.parameters.weight_decay",
    ]
].sort_values(by=("val/accuracy", "mean"), ascending=False)
# df_test = collect_subsets['PROTEINS'][[ 'test/accuracy', 'val/accuracy', 'model.model_name', 'model.model_domain', 'transforms.sann_encoding.neighborhoods', 'model.backbone.n_layers']].sort_values(by=('test/accuracy', 'mean'), ascending=False)
# df_test[df_test['model.model_name']=='HOPSE_GPSE']['model.model_domain'].unique()
# df_test.iloc[0]['transforms.sann_encoding.neighborhoods'].item()
df_test

test/accuracy                val/accuracy                model.model_name  \
             mean      std count         mean      std count                    
504       97.0051   1.2179     5      96.9087   1.1758     5  HOPSE_MANUAL_PE   
506       96.7659   1.0780     5      96.5100   0.9742     5  HOPSE_MANUAL_PE   
516       96.4951   1.5603     5      96.3635   1.4067     5  HOPSE_MANUAL_PE   
508       95.8832   1.4888     5      95.8201   1.3570     5  HOPSE_MANUAL_PE   
512       95.6124   1.6675     5      95.7126   1.3051     5  HOPSE_MANUAL_PE   
..            ...      ...   ...          ...      ...   ...              ...   
511       88.6064  13.9130     5      74.1103  35.8536     5  HOPSE_MANUAL_PE   
493       94.8299   0.0124     5      74.0121  38.5558     5  HOPSE_MANUAL_PE   
531       94.3978   0.9127     5      73.9713  35.8221     5  HOPSE_MANUAL_PE   
635       81.4984  29.3627     5      72.3041  35.1230     5  HOPSE_MANUAL_PE   
585       94.2142   1.0297     5      69.4613  35.0128     5  HOPSE_MANUAL_PE   

    transforms.sann_encoding.max_rank model.model_domain  \
                                                           
504                               2.0         simplicial   
506                               2.0         simplicial   
516                               2.0         simplicial   
508                               2.0         simplicial   
512                               2.0         simplicial   
..                                ...                ...   
511                               2.0         simplicial   
493                               2.0         simplicial   
531                               2.0         simplicial   
635                               2.0         simplicial   
585                               2.0         simplicial   

                transforms.sann_encoding.neighborhoods  \
                                                         
504  ['up_adjacency-0', 'up_adjacency-1', '2-up_adj...   
506  ['up_adjacency-0', 'up_adjacency-1', '2-up_adj...   
516  ['up_adjacency-0', 'up_adjacency-1', '2-up_adj...   
508  ['up_adjacency-0', 'up_adjacency-1', '2-up_adj...   
512  ['up_adjacency-0', 'up_adjacency-1', '2-up_adj...   
..                                                 ...   
511  ['up_adjacency-0', 'up_adjacency-1', '2-up_adj...   
493  ['up_adjacency-0', 'down_incidence-1', 'down_i...   
531  ['up_adjacency-0', 'up_adjacency-1', 'down_adj...   
635                                 ['up_adjacency-0']   
585  ['up_adjacency-0', 'up_incidence-0', 'up_incid...   

    model.backbone.n_layers optimizer.parameters.weight_decay  
                                                               
504                     1.0                            0.0000  
506                     1.0                            0.0001  
516                     2.0                            0.0000  
508                     1.0                            0.0000  
512                     2.0                            0.0000  
..                      ...                               ...  
511                     1.0                            0.0001  
493                     2.0                            0.0000  
531                     1.0                            0.0001  
635                     2.0                            0.0001  
585                     2.0                            0.0000  

[168 rows x 12 columns]

# Plot best models

In [13]:
def inc_list_to_name(inc_list):
    inc_list = eval(inc_list)
    inc_name = ""
    for inc in inc_list:
        inc_num = inc.split("-")
        inc_val = int(inc_num[0]) if len(inc_num) == 3 else 1
        dim = int(inc_num[-1])

        key = ""
        if "incidence" in inc:
            if "up" in inc:
                key = f"U_{dim}_{dim + inc_val}"
            elif "down" in inc:
                key = f"L_{dim - inc_val}_{dim}"
            else:
                raise Exception("Unknown NHBD")
        elif "adjacency" in inc:
            key = "A_"
            if "up" in inc:
                key = f"A_{dim}"
            elif "down" in inc:
                key = f"A_{dim}"
            else:
                raise Exception("Unknown NHBD")
        inc_name += key + ","
    return inc_name

In [74]:
for dataset in datasets:
    agg_sub = collect_subsets[dataset].copy()
    eval_metric = optimization_metrics[dataset]["eval_metric"]
    optim_dir = optimization_metrics[dataset]["direction"]
    print(dataset, optim_dir)
    agg_sub.sort_values(
        by=(eval_metric, "mean"), ascending=(optim_dir == "min"), inplace=True
    )
    agg_subset = agg_sub[:10].copy()

    cols = [
        "transforms.sann_encoding.neighborhoods",
        "transforms.sann_encoding.pretrain_model",
        "model.backbone.n_layers",
        "model.model_domain",
        "model.feature_encoder.proj_dropout",
        "model.feature_encoder.out_channels",
        "optimizer.parameters.weight_decay",
        "optimizer.parameters.lr",
        "dataset.dataloader_params.batch_size",
    ]

    # iterate over rows
    model_names = []
    for index, row in agg_subset.iterrows():
        m_name = row["model.model_name"].item()
        # Get values of the row
        values = [row[col].item() for col in cols]
        is_sann = "SANN" in m_name

        model_name = f"{m_name}|"
        for col, value in zip(cols, values):
            if col == "transforms.sann_encoding.max_hop" and is_sann:
                model_name += f"Hop={value}|"
            elif col == "model.backbone.n_layers":
                model_name += f"L={value}|"
            elif (
                col == "transforms.sann_encoding.neighborhoods" and not is_sann
            ):
                model_name += f"NH={inc_list_to_name(value)}|"
            elif col == "model.feature_encoder.out_channels":
                model_name += f"OCs={value}|"
            elif col == "optimizer.parameters.lr":
                model_name += f"LR={value}|"
            elif col == "optimizer.parameters.weight_decay":
                model_name += f"WD={value}|"
            elif col == "dataset.dataloader_params.batch_size":
                model_name += f"BS={value}|"
        model_names.append(model_name)
    agg_subset["Model_name"] = model_names

    # Plotting
    plt.figure(figsize=(12, 6))
    model_names = agg_subset["Model_name"]

    accuracy_means = np.array(agg_subset[(eval_metric, "mean")])
    accuracy_stds = np.array(agg_subset[(eval_metric, "std")])
    bars = plt.bar(
        model_names,
        accuracy_means,
        yerr=accuracy_stds,
        capsize=5,
        color="skyblue",
    )

    accuracy_means = np.array(agg_subset[(eval_metric, "mean")])
    # Adding data labels on top of bars
    for bar, mean in zip(bars, accuracy_means):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.2,
            f"{mean:.1f}",
            ha="center",
            va="bottom",
        )
    # plt.xticks(rotation=45)
    plt.title(f"Model performance for {dataset} dataset")
    plt.ylabel(f"Test {eval_metric}")
    plt.tight_layout()
    delta = agg_subset[(eval_metric, "std")].max() + 1
    plt.ylim(
        (agg_subset[(eval_metric, "mean")].min() - delta).round(),
        (agg_subset[(eval_metric, "mean")].max() + delta).round(),
    )
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    # Rotate x-axis labels
    plt.xticks(rotation=45, ha="right", fontsize=8)

    plt.savefig(
        f"figures/{dataset}_performance.png", dpi=300, bbox_inches="tight"
    )
    plt.close()

PROTEINS max
NCI109 max
NCI1 max
IMDB-BINARY max
ZINC min
IMDB-MULTI max


In [75]:
for dataset in datasets:
    aggregated = collect_subsets[dataset]
    for m_name in aggregated["model.model_name"].unique():
        agg_sub = aggregated[aggregated["model.model_name"] == m_name].copy()
        optim_metric = optimization_metrics[dataset]["eval_metric"]
        optim_dir = optimization_metrics[dataset]["direction"]
        agg_sub.sort_values(
            by=(optim_metric, "mean"),
            ascending=(optim_dir == "min"),
            inplace=True,
        )
        agg_subset = agg_sub[:10].copy()

        cols = [
            "transforms.sann_encoding.neighborhoods",
            "transforms.sann_encoding.pretrain_model",
            "model.backbone.n_layers",
            "model.model_domain",
            "model.feature_encoder.out_channels",
            "optimizer.parameters.weight_decay",
            "optimizer.parameters.lr",
            "model.feature_encoder.proj_dropout",
            "dataset.dataloader_params.batch_size",
        ]

        # iterate over rows
        model_names = []
        for index, row in agg_subset.iterrows():
            # Get values of the row
            values = [row[col].item() for col in cols]
            is_sann = "SANN" in m_name

            model_name = f"{m_name}|"
            for col, value in zip(cols, values):
                if col == "transforms.sann_encoding.max_hop" and is_sann:
                    model_name += f"Hop={value}|"
                elif col == "model.backbone.n_layers":
                    model_name += f"L={value}|"
                elif (
                    col == "transforms.sann_encoding.neighborhoods"
                    and not is_sann
                ):
                    model_name += f"NH={inc_list_to_name(value)}|"
                elif col == "model.feature_encoder.out_channels":
                    model_name += f"OCs={value}|"
                elif col == "optimizer.parameters.lr":
                    model_name += f"LR={value}|"
                elif col == "optimizer.parameters.weight_decay":
                    model_name += f"WD={value}|"
                elif col == "dataset.dataloader_params.batch_size":
                    model_name += f"BS={value}|"
            model_names.append(model_name)
        agg_subset["Model_name"] = model_names

        # Plotting
        plt.figure(figsize=(12, 6))
        model_names = agg_subset["Model_name"]

        accuracy_means = np.array(agg_subset[(optim_metric, "mean")])
        accuracy_stds = np.array(agg_subset[(optim_metric, "std")])
        bars = plt.bar(
            model_names,
            accuracy_means,
            yerr=accuracy_stds,
            capsize=5,
            color="skyblue",
        )

        accuracy_means = np.array(agg_subset[(optim_metric, "mean")])
        # Adding data labels on top of bars
        for bar, mean in zip(bars, accuracy_means):
            plt.text(
                bar.get_x() + bar.get_width() / 2,
                bar.get_height() + 0.2,
                f"{mean:.1f}",
                ha="center",
                va="bottom",
            )
        # plt.xticks(rotation=45)
        plt.title(f"Model performance for {dataset} dataset")
        plt.ylabel("Test accuracy")
        plt.tight_layout()
        delta = agg_subset[(optim_metric, "std")].max() + 1
        plt.ylim(
            (agg_subset[(optim_metric, "mean")].min() - delta).round(),
            (agg_subset[(optim_metric, "mean")].max() + delta).round(),
        )
        plt.grid(axis="y", linestyle="--", alpha=0.7)
        # Rotate x-axis labels
        plt.xticks(rotation=45, ha="right", fontsize=8)

        plt.savefig(
            f"figures/{m_name}_performance_{dataset}.png",
            dpi=300,
            bbox_inches="tight",
        )
        plt.close()

In [14]:
df_dict = {"model": [], "dataset": [], "mean": [], "std": [], "domain": []}

for dataset in datasets:
    aggregated = collect_subsets[dataset]
    for m_name in aggregated["model.model_name"].unique():
        for domain in aggregated["model.model_domain"].unique():
            agg_sub = aggregated[
                (aggregated["model.model_name"] == m_name)
                & (aggregated["model.model_domain"] == domain)
            ].copy()
            optim_metric = optimization_metrics[dataset]["optim_metric"]
            eval_metric = optimization_metrics[dataset]["eval_metric"]
            optim_dir = optimization_metrics[dataset]["direction"]
            agg_sub.sort_values(
                by=(optim_metric, "mean"),
                ascending=(optim_dir == "min"),
                inplace=True,
            )

            df_dict["domain"].append(
                agg_sub.iloc[0]["model.model_domain"].item()
            )
            df_dict["model"].append(m_name)
            df_dict["dataset"].append(dataset)
            df_dict["mean"].append(agg_sub.iloc[0][(eval_metric, "mean")])
            df_dict["std"].append(agg_sub.iloc[0][(eval_metric, "std")])
df_res = pd.DataFrame(df_dict)

In [67]:
data = []
data.extend(
    [
        {
            "model": "GCCN with GAT",
            "domain": "cell",
            "dataset": "MUTAG",
            "mean": 83.40,
            "std": 4.85,
        },
        {
            "model": "GCCN with GAT",
            "domain": "cell",
            "dataset": "PROTEINS",
            "mean": 74.05,
            "std": 2.16,
        },
        {
            "model": "GCCN with GAT",
            "domain": "cell",
            "dataset": "NCI1",
            "mean": 76.11,
            "std": 1.69,
        },
        {
            "model": "GCCN with GAT",
            "domain": "cell",
            "dataset": "NCI109",
            "mean": 75.62,
            "std": 0.76,
        },
        {
            "model": "GCCN with GAT",
            "domain": "cell",
            "dataset": "ZINC",
            "mean": 0.38,
            "std": 0.03,
        },
        # {'model': 'Cell with GAT', 'domain': 'cell', 'dataset': 'Cora', 'mean': 88.39, 'std': 0.65},
        # {'model': 'Cell with GAT', 'domain': 'cell', 'dataset': 'Citeseer', 'mean': 74.62, 'std': 1.95},
        # {'model': 'Cell with GAT', 'domain': 'cell', 'dataset': 'PubMed', 'mean': 87.68, 'std': 0.33},
        {
            "model": "GCCN with GCN",
            "domain": "cell",
            "dataset": "MUTAG",
            "mean": 85.11,
            "std": 6.73,
        },
        {
            "model": "GCCN with GCN",
            "domain": "cell",
            "dataset": "PROTEINS",
            "mean": 74.41,
            "std": 1.77,
        },
        {
            "model": "GCCN with GCN",
            "domain": "cell",
            "dataset": "NCI1",
            "mean": 76.42,
            "std": 1.67,
        },
        {
            "model": "GCCN with GCN",
            "domain": "cell",
            "dataset": "NCI109",
            "mean": 75.62,
            "std": 0.94,
        },
        {
            "model": "GCCN with GCN",
            "domain": "cell",
            "dataset": "ZINC",
            "mean": 0.36,
            "std": 0.01,
        },
        # {'model': 'Cell with GCN', 'domain': 'cell', 'dataset': 'Cora', 'mean': 88.51, 'std': 0.70},
        # {'model': 'Cell with GCN', 'domain': 'cell', 'dataset': 'Citeseer', 'mean': 75.41, 'std': 2.00},
        # {'model': 'Cell with GCN', 'domain': 'cell', 'dataset': 'PubMed', 'mean': 88.18, 'std': 0.26},
        {
            "model": "GCCN with GIN",
            "domain": "cell",
            "dataset": "MUTAG",
            "mean": 86.38,
            "std": 6.49,
        },
        {
            "model": "GCCN with GIN",
            "domain": "cell",
            "dataset": "PROTEINS",
            "mean": 72.54,
            "std": 3.07,
        },
        {
            "model": "GCCN with GIN",
            "domain": "cell",
            "dataset": "NCI1",
            "mean": 77.65,
            "std": 1.11,
        },
        {
            "model": "GCCN with GIN",
            "domain": "cell",
            "dataset": "NCI109",
            "mean": 77.19,
            "std": 0.21,
        },
        {
            "model": "GCCN with GIN",
            "domain": "cell",
            "dataset": "ZINC",
            "mean": 0.19,
            "std": 0.00,
        },
        # {'model': 'Cell with GIN', 'domain': 'cell', 'dataset': 'Cora', 'mean': 87.42, 'std': 1.85},
        # {'model': 'Cell with GIN', 'domain': 'cell', 'dataset': 'Citeseer', 'mean': 75.13, 'std': 1.17},
        # {'model': 'Cell with GIN', 'domain': 'cell', 'dataset': 'PubMed', 'mean': 88.47, 'std': 0.27},
        {
            "model": "GCCN with GraphSAGE",
            "domain": "cell",
            "dataset": "MUTAG",
            "mean": 85.53,
            "std": 6.80,
        },
        {
            "model": "GCCN with GraphSAGE",
            "domain": "cell",
            "dataset": "PROTEINS",
            "mean": 73.62,
            "std": 2.72,
        },
        {
            "model": "GCCN with GraphSAGE",
            "domain": "cell",
            "dataset": "NCI1",
            "mean": 78.23,
            "std": 1.47,
        },
        {
            "model": "GCCN with GraphSAGE",
            "domain": "cell",
            "dataset": "NCI109",
            "mean": 77.10,
            "std": 0.83,
        },
        {
            "model": "GCCN with GraphSAGE",
            "domain": "cell",
            "dataset": "ZINC",
            "mean": 0.24,
            "std": 0.00,
        },
        # {'model': 'Cell with GraphSAGE', 'domain': 'cell', 'dataset': 'Cora', 'mean': 88.57, 'std': 0.58},
        # {'model': 'Cell with GraphSAGE', 'domain': 'cell', 'dataset': 'Citeseer', 'mean': 75.89, 'std': 1.84},
        # {'model': 'Cell with GraphSAGE', 'domain': 'cell', 'dataset': 'PubMed', 'mean': 89.40, 'std': 0.57},
        {
            "model": "GCCN with Transformer",
            "domain": "cell",
            "dataset": "MUTAG",
            "mean": 83.83,
            "std": 6.49,
        },
        {
            "model": "GCCN with Transformer",
            "domain": "cell",
            "dataset": "PROTEINS",
            "mean": 70.97,
            "std": 4.06,
        },
        {
            "model": "GCCN with Transformer",
            "domain": "cell",
            "dataset": "NCI1",
            "mean": 73.00,
            "std": 1.37,
        },
        {
            "model": "GCCN with Transformer",
            "domain": "cell",
            "dataset": "NCI109",
            "mean": 73.20,
            "std": 1.05,
        },
        {
            "model": "GCCN with Transformer",
            "domain": "cell",
            "dataset": "ZINC",
            "mean": 0.45,
            "std": 0.02,
        },
        # {'model': 'Cell with Transformer', 'domain': 'cell', 'dataset': 'Cora', 'mean': 84.61, 'std': 1.32},
        # {'model': 'Cell with Transformer', 'domain': 'cell', 'dataset': 'Citeseer', 'mean': 75.05, 'std': 1.67},
        # {'model': 'Cell with Transformer', 'domain': 'cell', 'dataset': 'PubMed', 'mean': 88.37, 'std': 0.22},
        {
            "model": "GCCN with Hasse",
            "domain": "cell",
            "dataset": "MUTAG",
            "mean": 85.96,
            "std": 7.15,
        },
        {
            "model": "GCCN with Hasse",
            "domain": "cell",
            "dataset": "PROTEINS",
            "mean": 73.73,
            "std": 2.95,
        },
        {
            "model": "GCCN with Hasse",
            "domain": "cell",
            "dataset": "NCI1",
            "mean": 76.75,
            "std": 1.63,
        },
        {
            "model": "GCCN with Hasse",
            "domain": "cell",
            "dataset": "NCI109",
            "mean": 76.94,
            "std": 0.82,
        },
        {
            "model": "GCCN with Hasse",
            "domain": "cell",
            "dataset": "ZINC",
            "mean": 0.31,
            "std": 0.01,
        },
        # {'model': 'Cell with Hasse', 'domain': 'cell', 'dataset': 'Cora', 'mean': 87.24, 'std': 0.58},
        # {'model': 'Cell with Hasse', 'domain': 'cell', 'dataset': 'Citeseer', 'mean': 74.26, 'std': 1.47},
        # {'model': 'Cell with Hasse', 'domain': 'cell', 'dataset': 'PubMed', 'mean': 88.65, 'std': 0.55},
    ]
)

# Simplicial models
data.extend(
    [
        {
            "model": "GCCN with GAT",
            "domain": "simplicial",
            "dataset": "MUTAG",
            "mean": 79.15,
            "std": 4.09,
        },
        {
            "model": "GCCN with GAT",
            "domain": "simplicial",
            "dataset": "PROTEINS",
            "mean": 74.62,
            "std": 1.95,
        },
        {
            "model": "GCCN with GAT",
            "domain": "simplicial",
            "dataset": "NCI1",
            "mean": 74.86,
            "std": 1.42,
        },
        {
            "model": "GCCN with GAT",
            "domain": "simplicial",
            "dataset": "NCI109",
            "mean": 74.81,
            "std": 1.14,
        },
        {
            "model": "GCCN with GAT",
            "domain": "simplicial",
            "dataset": "ZINC",
            "mean": 0.57,
            "std": 0.03,
        },
        # {'model': 'GCCN with GAT', 'domain': 'simplicial', 'dataset': 'Cora', 'mean': 88.33, 'std': 0.67},
        # {'model': 'GCCN with GAT', 'domain': 'simplicial', 'dataset': 'Citeseer', 'mean': 74.65, 'std': 1.93},
        # {'model': 'GCCN with GAT', 'domain': 'simplicial', 'dataset': 'PubMed', 'mean': 87.72, 'std': 0.36},
        {
            "model": "GCCN with GCN",
            "domain": "simplicial",
            "dataset": "MUTAG",
            "mean": 74.04,
            "std": 8.30,
        },
        {
            "model": "GCCN with GCN",
            "domain": "simplicial",
            "dataset": "PROTEINS",
            "mean": 74.91,
            "std": 2.51,
        },
        {
            "model": "GCCN with GCN",
            "domain": "simplicial",
            "dataset": "NCI1",
            "mean": 74.20,
            "std": 2.17,
        },
        {
            "model": "GCCN with GCN",
            "domain": "simplicial",
            "dataset": "NCI109",
            "mean": 74.13,
            "std": 0.53,
        },
        {
            "model": "GCCN with GCN",
            "domain": "simplicial",
            "dataset": "ZINC",
            "mean": 0.53,
            "std": 0.05,
        },
        # {'model': 'GCN', 'domain': 'simplicial', 'dataset': 'Cora', 'mean': 88.51, 'std': 0.70},
        # {'model': 'GCN', 'domain': 'simplicial', 'dataset': 'Citeseer', 'mean': 75.41, 'std': 2.00},
        # {'model': 'GCN', 'domain': 'simplicial', 'dataset': 'PubMed', 'mean': 88.19, 'std': 0.24},
        {
            "model": "GCCN with GIN",
            "domain": "simplicial",
            "dataset": "MUTAG",
            "mean": 85.96,
            "std": 4.66,
        },
        {
            "model": "GCCN with GIN",
            "domain": "simplicial",
            "dataset": "PROTEINS",
            "mean": 72.83,
            "std": 2.72,
        },
        {
            "model": "GCCN with GIN",
            "domain": "simplicial",
            "dataset": "NCI1",
            "mean": 76.67,
            "std": 1.62,
        },
        {
            "model": "GCCN with GIN",
            "domain": "simplicial",
            "dataset": "NCI109",
            "mean": 75.76,
            "std": 1.28,
        },
        {
            "model": "GCCN with GIN",
            "domain": "simplicial",
            "dataset": "ZINC",
            "mean": 0.35,
            "std": 0.01,
        },
        # {'model': 'GIN', 'domain': 'simplicial', 'dataset': 'Cora', 'mean': 87.27, 'std': 1.63},
        # {'model': 'GIN', 'domain': 'simplicial', 'dataset': 'Citeseer', 'mean': 75.05, 'std': 1.27},
        # {'model': 'GIN', 'domain': 'simplicial', 'dataset': 'PubMed', 'mean': 88.54, 'std': 0.21},
        {
            "model": "GCCN with GraphSAGE",
            "domain": "simplicial",
            "dataset": "MUTAG",
            "mean": 75.74,
            "std": 2.43,
        },
        {
            "model": "GCCN with GraphSAGE",
            "domain": "simplicial",
            "dataset": "PROTEINS",
            "mean": 74.70,
            "std": 3.10,
        },
        {
            "model": "GCCN with GraphSAGE",
            "domain": "simplicial",
            "dataset": "NCI1",
            "mean": 76.85,
            "std": 1.50,
        },
        {
            "model": "GCCN with GraphSAGE",
            "domain": "simplicial",
            "dataset": "NCI109",
            "mean": 75.64,
            "std": 1.94,
        },
        {
            "model": "GCCN with GraphSAGE",
            "domain": "simplicial",
            "dataset": "ZINC",
            "mean": 0.50,
            "std": 0.02,
        },
        # {'model': 'GraphSAGE', 'domain': 'simplicial', 'dataset': 'Cora', 'mean': 88.57, 'std': 0.59},
        # {'model': 'GraphSAGE', 'domain': 'simplicial', 'dataset': 'Citeseer', 'mean': 75.92, 'std': 1.85},
        # {'model': 'GraphSAGE', 'domain': 'simplicial', 'dataset': 'PubMed', 'mean': 89.34, 'std': 0.39},
        {
            "model": "GCCN with Transformer",
            "domain": "simplicial",
            "dataset": "MUTAG",
            "mean": 74.04,
            "std": 4.09,
        },
        {
            "model": "GCCN with Transformer",
            "domain": "simplicial",
            "dataset": "PROTEINS",
            "mean": 70.97,
            "std": 4.06,
        },
        {
            "model": "GCCN with Transformer",
            "domain": "simplicial",
            "dataset": "NCI1",
            "mean": 70.39,
            "std": 0.96,
        },
        {
            "model": "GCCN with Transformer",
            "domain": "simplicial",
            "dataset": "NCI109",
            "mean": 69.99,
            "std": 1.13,
        },
        {
            "model": "GCCN with Transformer",
            "domain": "simplicial",
            "dataset": "ZINC",
            "mean": 0.64,
            "std": 0.01,
        },
        # {'model': 'Transformer', 'domain': 'simplicial', 'dataset': 'Cora', 'mean': 84.40, 'std': 1.16},
        # {'model': 'Transformer', 'domain': 'simplicial', 'dataset': 'Citeseer', 'mean': 74.60, 'std': 1.88},
        # {'model': 'Transformer', 'domain': 'simplicial', 'dataset': 'PubMed', 'mean': 88.55, 'std': 0.39},
        {
            "model": "GCCN with Hasse",
            "domain": "simplicial",
            "dataset": "MUTAG",
            "mean": 74.04,
            "std": 5.51,
        },
        {
            "model": "GCCN with Hasse",
            "domain": "simplicial",
            "dataset": "PROTEINS",
            "mean": 74.48,
            "std": 1.89,
        },
        {
            "model": "GCCN with Hasse",
            "domain": "simplicial",
            "dataset": "NCI1",
            "mean": 75.02,
            "std": 2.24,
        },
        {
            "model": "GCCN with Hasse",
            "domain": "simplicial",
            "dataset": "NCI109",
            "mean": 73.91,
            "std": 3.90,
        },
        {
            "model": "GCCN with Hasse",
            "domain": "simplicial",
            "dataset": "ZINC",
            "mean": 0.56,
            "std": 0.02,
        },
        # {'model': 'Simplicial with Hasse', 'domain': 'simplicial', 'dataset': 'Cora', 'mean': 87.56, 'std': 0.66},
        # {'model': 'Simplicial with Hasse', 'domain': 'simplicial', 'dataset': 'Citeseer', 'mean': 74.50, 'std': 1.61},
        # {'model': 'Simplicial with Hasse', 'domain': 'simplicial', 'dataset': 'PubMed', 'mean': 88.61, 'std': 0.27},
    ]
)
topotune_df = pd.DataFrame(data)

In [68]:
# Define the raw table data
raw_table_data = {
    # Format: {dataset: [(method, mean, std), ...]}
    # 'Cora': [
    #     ('CWN', 74.95, 0.98),
    #     ('CCCN', 86.32, 1.38),
    #     ('SCCNN', 87.44, 1.17),
    #     ('SCN', 87.68, 1.17),
    #     ('DR_cell', 82.19, 1.07),
    #     ('SDP_cell', 80.65, 2.39),
    #     ('DR_simplicial', 82.27, 1.34),
    #     ('SDP_simplicial', 79.91, 1.18)
    # ],
    # 'Citeseer': [
    #     ('CWN', 70.49, 2.85),
    #     ('CCCN', 75.20, 1.82),
    #     ('SCCNN', 75.63, 1.58),
    #     ('SCN', 74.91, 1.25),
    #     ('DR_cell', 70.23, 2.69),
    #     ('SDP_cell', 69.03, 2.01),
    #     ('DR_simplicial', 71.24, 1.68),
    #     ('SDP_simplicial', 70.40, 1.53)
    # ],
    # 'PubMed': [
    #     ('CWN', 86.94, 0.68),
    #     ('CCCN', 88.64, 0.36),
    #     ('SCCNN', 88.52, 0.44),
    #     ('SCN', 88.67, 0.39),
    #     ('DR_cell', 88.18, 0.32),
    #     ('SDP_cell', 87.78, 0.58),
    #     ('DR_simplicial', 88.72, 0.50),
    #     ('SDP_simplicial', 88.62, 0.44)
    # ],
    "MUTAG": [
        ("CWN", 69.68, 8.58),
        ("CCCN", 80.43, 1.78),
        ("SCCNN", 80.85, 5.42),
        ("SCN", 77.02, 9.32),
        ("DR_cell", 76.17, 6.63),
        ("SDP_cell", 70.64, 3.16),
        ("DR_simplicial", 71.49, 2.43),
        ("SDP_simplicial", 73.62, 6.13),
    ],
    "PROTEINS": [
        ("CWN", 76.13, 1.80),
        ("CCCN", 76.13, 2.70),
        ("SCCNN", 73.55, 3.43),
        ("SCN", 73.33, 2.30),
        ("DR_cell", 74.19, 2.86),
        ("SDP_cell", 74.98, 1.92),
        ("DR_simplicial", 75.27, 2.14),
        ("SDP_simplicial", 74.77, 1.69),
    ],
    "NCI1": [
        ("CWN", 68.52, 0.51),
        ("CCCN", 73.93, 1.87),
        ("SCCNN", 76.67, 1.48),
        ("SCN", 77.65, 1.28),
        ("DR_cell", 76.60, 1.75),
        ("SDP_cell", 75.60, 2.45),
        ("DR_simplicial", 75.27, 1.57),
        ("SDP_simplicial", 74.49, 1.03),
    ],
    "NCI109": [
        ("CWN", 68.19, 0.65),
        ("CCCN", 73.80, 2.06),
        ("SCCNN", 75.35, 1.50),
        ("SCN", 74.83, 1.18),
        ("DR_cell", 77.12, 1.07),
        ("SDP_cell", 75.43, 1.94),
        ("DR_simplicial", 74.58, 1.29),
        ("SDP_simplicial", 75.70, 1.04),
    ],
    "ZINC": [
        ("CWN", 0.70, 0.00),
        ("CCCN", 0.34, 0.01),
        ("SCCNN", 0.35, 0.02),
        ("SCN", 0.34, 0.02),
        ("DR_cell", 0.36, 0.01),
        ("SDP_cell", 0.36, 0.02),
        ("DR_simplicial", 0.59, 0.01),
        ("SDP_simplicial", 0.53, 0.04),
    ],
}


# Process the data to select the best of DR and SDP for each method
additional_data = []

for dataset, entries in raw_table_data.items():
    optim_dir = optimization_metrics[dataset]["direction"]

    # Group data by method prefix (before the underscore)
    method_results = {}
    standard_methods = []
    dr_sdp_methods = {"cell": {}, "simplicial": {}}

    for method, mean, std in entries:
        if method in ["CWN", "CCCN", "SCCNN", "SCN"]:
            # These are the standard methods - just add them directly
            standard_methods.append(
                {
                    "method": method,
                    "dataset": dataset,
                    "mean": mean,
                    "std": std,
                }
            )
        elif method.startswith("DR_") or method.startswith("SDP_"):
            # These are DR or SDP methods - group by domain
            method_type, domain = method.split("_")
            dr_sdp_methods[domain][method_type] = {"mean": mean, "std": std}

    # Add all standard methods
    additional_data.extend(standard_methods)

    # Select the best of DR/SDP for each domain
    for domain in ["cell", "simplicial"]:
        if "DR" in dr_sdp_methods[domain] and "SDP" in dr_sdp_methods[domain]:
            dr_result = dr_sdp_methods[domain]["DR"]
            sdp_result = dr_sdp_methods[domain]["SDP"]

            # Determine which is better based on optimization direction
            if optim_dir == "max":
                if dr_result["mean"] >= sdp_result["mean"]:
                    best_method = "DR"
                    best_result = dr_result
                else:
                    best_method = "SDP"
                    best_result = sdp_result
            else:  # min direction
                if dr_result["mean"] <= sdp_result["mean"]:
                    best_method = "DR"
                    best_result = dr_result
                else:
                    best_method = "SDP"
                    best_result = sdp_result

            # Add the best result
            method_name = f"{best_method}_{domain}"
            additional_data.append(
                {
                    "method": method_name,
                    "dataset": dataset,
                    "mean": best_result["mean"],
                    "std": best_result["std"],
                }
            )


# Map method names to their proper format in the data
method_mappings = {
    "CWN": "Cell CWN",
    "CCCN": "CCNN",  # Cell CCNN
    "SCCNN": "Simplicial CCNN",
    "SCN": "Simplicial SCN",
    "DR_cell": "Cell DR",
    "SDP_cell": "Cell SDP",
    "DR_simplicial": "DR",
    "SDP_simplicial": "SDP",
}

# Domain mappings
domain_mappings = {
    "CWN": "cell",
    "CCCN": "cell",
    "SCCNN": "simplicial",
    "SCN": "simplicial",
    "DR_cell": "cell",
    "SDP_cell": "cell",
    "DR_simplicial": "simplicial",
    "SDP_simplicial": "simplicial",
}

# For display purposes, we'll use simplified method names
display_mappings = {
    "DR_cell": "DR",
    "SDP_cell": "SDP",
    "DR_simplicial": "DR",
    "SDP_simplicial": "SDP",
}

# Convert the additional data to the proper format for the dataframe
formatted_data = []
for item in additional_data:
    formatted_data.append(
        {
            "model": method_mappings[item["method"]],
            "domain": domain_mappings[item["method"]],
            "dataset": item["dataset"],
            "mean": item["mean"],
            "std": item["std"],
        }
    )

# This data can now be added to your existing dataframe or used to create a new one
tbx_df = pd.DataFrame(formatted_data)

In [69]:
final_add_df = pd.concat([df_res, topotune_df, tbx_df], ignore_index=True)

In [70]:
final_add_df[
    final_add_df.duplicated(subset=["model", "dataset", "domain"], keep=False)
]

,model,dataset,mean,std,domain


In [63]:
def generate_latex_table(df):
    """
    Generates a LaTeX table with methods as rows and datasets as columns.
    For models containing "with", splits into first word (model) and last word (variant).

    Parameters:
    df (pandas.DataFrame): DataFrame with columns 'model', 'domain', 'dataset', 'mean', and 'std'

    Returns:
    tuple: (LaTeX preamble, LaTeX table code)
    """
    # Ensure df has required columns
    required_columns = ["model", "domain", "dataset", "mean", "std"]
    for col in required_columns:
        if col not in df.columns:
            raise ValueError("DataFrame must contain column: {}".format(col))

    # Remove duplicates and show warning if any are found
    duplicates = df.duplicated(
        subset=["model", "domain", "dataset"], keep=False
    )
    if duplicates.any():
        print(
            "Warning: Found {} duplicate entries. Using the first occurrence.".format(
                duplicates.sum()
            )
        )
        df = df.drop_duplicates(
            subset=["model", "domain", "dataset"], keep="first"
        )

    # Instead of pivot, create a dictionary structure directly
    data_dict = {}

    # Group by domain, then model, then dataset
    for _, row in df.iterrows():
        domain = row["domain"]
        model = row["model"]
        dataset = row["dataset"]
        mean = row["mean"]
        std = row["std"]

        # Initialize nested dictionaries if they don't exist
        if domain not in data_dict:
            data_dict[domain] = {}
        if model not in data_dict[domain]:
            data_dict[domain][model] = {}

        # Store the mean and std values
        data_dict[domain][model][dataset] = {"mean": mean, "std": std}

    # Get sorted lists of unique values
    domains = sorted(data_dict.keys())
    datasets = sorted(df["dataset"].unique())

    # Find best results for each dataset
    best_results = {}

    for dataset in datasets:
        all_means = []
        for domain in domains:
            for model in data_dict[domain]:
                if dataset in data_dict[domain][model]:
                    all_means.append(data_dict[domain][model][dataset]["mean"])

        if all_means:
            optim_dir = optimization_metrics.get(
                dataset, {"direction": "max"}
            )["direction"]
            if optim_dir == "max":
                best_results[dataset] = max(all_means)
            else:
                best_results[dataset] = min(all_means)

    # Helper function to process model names for display
    def process_model_name(model_name, domain):
        # Remove domain prefix if present
        domain_prefix = domain.capitalize() + " "
        if model_name.startswith(domain_prefix):
            model_name = model_name[len(domain_prefix) :]

        # For models with "with", split into main model and variant
        if " with " in model_name:
            parts = model_name.split()
            main_model = parts[0]  # First word (usually GCCN)
            variant = parts[-1]  # Last word after "with"
            return main_model, variant
        elif "GPSE" in model_name:
            # Special handling for GPSE
            return "GPSE", model_name.replace("GPSE", "").strip()
        else:
            # No splitting needed
            return model_name, ""

    # Function to generate a LaTeX row for a model
    def generate_model_row(
        model, domain, is_first_in_group=False, group_name=""
    ):
        # Process model name
        main_model, variant = process_model_name(model, domain)

        # Start row
        if is_first_in_group:
            row = "\\multirow{{1}}{{*}}{{{}}} & ".format(group_name)
        else:
            row = " & "

        # Add variant name
        row += "\\emph{{{}}}".format(variant if variant else main_model)

        # Add data for each dataset
        for dataset in datasets:
            if dataset in data_dict[domain][model]:
                mean = data_dict[domain][model][dataset]["mean"]
                std = data_dict[domain][model][dataset]["std"]

                # Format based on whether it's the best result
                if dataset in best_results and np.isclose(
                    mean, best_results[dataset]
                ):
                    row += " & \\cellcolor{{gray!30}}\\textbf{{{0:.3f}}} \\scriptsize{{±{1:.3f}}}".format(
                        mean, std
                    )
                else:
                    # Check if within one standard deviation of best
                    optim_dir = optimization_metrics.get(
                        dataset, {"direction": "max"}
                    )["direction"]
                    threshold = (
                        best_results[dataset] - std
                        if dataset in best_results
                        else None
                    )

                    if threshold is not None:
                        if (optim_dir == "max" and mean >= threshold) or (
                            optim_dir == "min" and mean <= threshold
                        ):
                            row += " & \\cellcolor{{blue!20}}{0:.3f} \\scriptsize{{±{1:.3f}}}".format(
                                mean, std
                            )
                        else:
                            row += (
                                " & {0:.3f} \\scriptsize{{±{1:.3f}}}".format(
                                    mean, std
                                )
                            )
                    else:
                        row += " & {0:.3f} \\scriptsize{{±{1:.3f}}}".format(
                            mean, std
                        )
            else:
                row += " & -"

        return row + " \\\\"

    # Generate LaTeX table
    latex_table = []

    # Process each domain
    for domain in domains:
        # Organize models by main type
        domain_models = {}

        # Group models based on their main type (after processing)
        for model in data_dict[domain]:
            main_type, _ = process_model_name(model, domain)

            if main_type not in domain_models:
                domain_models[main_type] = []

            domain_models[main_type].append(model)

        # Sort model groups and models within groups
        sorted_types = sorted(domain_models.keys())
        for model_type in sorted_types:
            domain_models[model_type] = sorted(domain_models[model_type])

        # Add domain header
        domain_display = domain.capitalize()
        latex_table.append(
            "\\multicolumn{{{0}}}{{l}}{{\\textbf{{{1} Methods}}}} \\\\".format(
                len(datasets) + 2, domain_display
            )
        )
        latex_table.append("\\midrule")

        # Process each model type group
        for i, model_type in enumerate(sorted_types):
            models = domain_models[model_type]

            # Add group header with multirow
            latex_table.append(
                "\\multirow{{{0}}}{{*}}{{{1}}} & ".format(
                    len(models), model_type
                )
            )

            # Process each model in the group
            for j, model in enumerate(models):
                if j == 0:
                    # First model in group, header already added
                    row = generate_model_row(
                        model,
                        domain,
                        is_first_in_group=True,
                        group_name=model_type,
                    )
                else:
                    # Subsequent models in group
                    row = (
                        " & "
                        + generate_model_row(model, domain).split(" & ", 1)[1]
                    )

                latex_table.append(row)

                # Add cmidrule between models but not after the last one
                if j < len(models) - 1:
                    latex_table.append(
                        "\\cmidrule{{2-{0}}}".format(len(datasets) + 2)
                    )

            # Add midrule between different model types but not after the last one
            if i < len(sorted_types) - 1:
                latex_table.append("\\midrule")

        # Add midrule between domains
        if domain != domains[-1]:
            latex_table.append("\\midrule")

    # Construct the LaTeX table header
    dataset_labels = " & & " + " & ".join(
        ["\\emph{{{}}}".format(dataset) for dataset in datasets]
    )

    # Additional packages needed
    preamble = "\\usepackage{multirow}\n\\usepackage{booktabs}\n\\usepackage{colortbl}\n\\usepackage{array}"

    # Create column specifications
    col_spec = (
        ">{{\\raggedright\\arraybackslash}}p{{2.5cm}}>{{\\raggedright\\arraybackslash}}p{{2.5cm}}"
        + "c" * len(datasets)
    )

    latex_code = (
        "\\begin{table}[h]\n"
        "\\centering\n"
        "\\scriptsize\n"
        "\\begin{tabular}{" + col_spec + "}\n"
        "\\toprule\n"
        "\\textbf{Method} & \\textbf{Variant} "
        + dataset_labels
        + " \\\\\n"
        + "\n".join(latex_table)
        + "\n\\bottomrule\n"
        "\\end{tabular}\n"
        "\\caption{Cross-domain comparison grouped by domain type (Simplicial/Cellular): results are shown as mean and standard deviation. "
        "The best result is bold and shaded in grey, while those within one standard deviation are in blue-shaded boxes.}\n"
        "\\end{table}"
    )

    return preamble, latex_code

In [64]:
print(generate_latex_table(final_add_df)[1])

\begin{table}[h]
\centering
\scriptsize
\begin{tabular}{>{{\raggedright\arraybackslash}}p{{2.5cm}}>{{\raggedright\arraybackslash}}p{{2.5cm}}ccccccc}
\toprule
\textbf{Method} & \textbf{Variant}  & & \emph{IMDB-BINARY} & \emph{IMDB-MULTI} & \emph{MUTAG} & \emph{NCI1} & \emph{NCI109} & \emph{PROTEINS} & \emph{ZINC} \\
\multicolumn{9}{l}{\textbf{Cell Methods}} \\
\midrule
\multirow{4}{*}{GPSE} & 
\emph{_GEOM} & \cellcolor{gray!30}\textbf{73.200} \scriptsize{±1.876} & \cellcolor{blue!20}49.013 \scriptsize{±3.783} & - & 78.016 \scriptsize{±1.175} & \cellcolor{blue!20}77.909 \scriptsize{±1.246} & \cellcolor{blue!20}74.767 \scriptsize{±3.695} & 0.210 \scriptsize{±0.002} \\
\cmidrule{2-9}
 & \emph{_MOLPCBA} & \cellcolor{blue!20}70.960 \scriptsize{±5.104} & \cellcolor{gray!30}\textbf{50.027} \scriptsize{±3.561} & - & \cellcolor{blue!20}78.521 \scriptsize{±0.947} & 77.483 \scriptsize{±0.641} & \cellcolor{blue!20}74.839 \scriptsize{±2.590} & 0.213 \scriptsize{±0.005} \\
\cmidrule{2-9}
 & \emph{_PC